Выбор большой языковой модели

In [1]:
from llamba.chatmodels.chatbase import ChatbaseModel
import config

chatbot = ChatbaseModel(url=config.URL_CB, api_key=config.API_KEY_CB, chatbot_id=config.ID_CB)

Подготовка данных

In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np

from llamba_library.bioage_model import BioAgeModel
from llamba_library.functions import get_shap_dict
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_SHAP, FN_CHECKPOINT


#### Данные

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

data = pd.DataFrame(my_data, index=[0])


#### Модель
    
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(fn_model)
bioage_model = BioAgeModel(model=model)

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
shap_dict = get_shap_dict(FN_SHAP)
explainer = shap_dict['explainer']
feats = data.drop(['Age'], axis=1).columns.to_list()

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint H:\Lobachevsky\txai_omics_3\data\immuno\model.ckpt`
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


Составление запроса для экспертной системы 

In [3]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)
prompts = connector.generate_prompts(n=2, data=data, feats=feats, explainer=explainer) # n - количество признаков, внесших наибольший вклад
print(prompts)

['What is CXCL9? What does an increased level of CXCL9 mean?', 'What is CD40LG? What does an increased level of CD40LG mean?']


Передача экспертной системе данных 

In [4]:
for prompt in prompts:
    print(prompt)
    res = connector.query_prompt(prompt)
    print(res)

What is CXCL9? What does an increased level of CXCL9 mean?
CXCL9 представляет собой провоспалительный цитокин, который опосредует воспалительные реакции и связан с риском смертности от сердечно-сосудистых заболеваний. Увеличение уровня CXCL9 может указывать на участие в системном хроническом воспалении, связанном со старением сердечно-сосудистой системы и нарушением эластичности сосудов.
What is CD40LG? What does an increased level of CD40LG mean?
Hmm, I am not sure.
